Расмотрим пример на датасете из репозитория UCI

Описание данных - https://archive.ics.uci.edu/ml/datasets/Blood+Transfusion+Service+Center

In [9]:
import pandas as pd
import numpy as np
data = pd.read_csv("D:/Users/aavramenko/Downloads/transfusion.data", header=None)
data.head(3)

,0,1,2,3,4
0,2,50,12500,98,1
1,0,13,3250,28,1
2,1,16,4000,35,1


У нас есть 4 признака и 1 целевая переменная (бинарная) - нужно определить сдавал ли он/она кровь в марте 2007 г. (1 означает сдачу крови; 0 означает отсутствие сдачи крови).

In [21]:
print(data.shape)

(748, 5)


Всего 748 доноров

Посмотрим на соотношение классов

In [22]:
data.iloc[:, -1].value_counts()

0    570
1    178
Name: 4, dtype: int64

Разбиваем выборку на тренировочную и тестовую части и обучаем модель (в примере - градиентный бустинг)

In [23]:
from sklearn.model_selection import train_test_split

x_data = data.iloc[:,:-1]
y_data = data.iloc[:,-1]

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=7)

In [24]:
import xgboost as xgb

model = xgb.XGBClassifier()

model.fit(x_train, y_train)
y_predict = model.predict(x_test)

Проверяем качество

In [25]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 

    
evaluate_results(y_test, y_predict)

Classification results:
f1: 31.75%
roc: 56.46%
recall: 26.32%
precision: 40.00%


### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [26]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# leave just 25% of the positives marked
pos_sample_len = int(np.ceil(0.25 * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 45/178 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [27]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    703
 1     45
Name: class_test, dtype: int64


In [28]:
mod_data.head(10)

,0,1,2,3,4,class_test
0,2,50,12500,98,1,-1
1,0,13,3250,28,1,-1
2,1,16,4000,35,1,1
3,2,20,5000,45,1,1
4,1,24,6000,77,0,-1
5,4,4,1000,4,0,-1
6,2,7,1750,14,1,-1
7,1,12,3000,35,0,-1
8,2,9,2250,22,1,-1
9,5,46,11500,98,1,-1


In [29]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

### 1. random negative sampling

In [30]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(45, 6) (45, 6)


In [31]:
model = xgb.XGBClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)
evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 36.76%
roc: 61.01%
recall: 68.85%
precision: 25.07%
